In [7]:
import pickle as pkl

In [8]:
with open("results/bc/scores.pkl", "rb") as file:
    scores_list = pkl.load(file=file)

In [9]:
scores_list

[{'nona euclidean': [189, 0.10895061492919922],
  'nona euclidean mlp': [192, 0.7034478187561035],
  'tuned xgb': [187, 126.74830627441406],
  'tuned knn': [189, 0.8539342880249023]},
 {'nona euclidean': [188, 0.0008270740509033203],
  'nona euclidean mlp': [192, 0.4064502716064453],
  'tuned xgb': [190, 132.34864354133606],
  'tuned knn': [192, 0.752582311630249]},
 {'nona euclidean': [188, 0.0008399486541748047],
  'nona euclidean mlp': [194, 0.5216686725616455],
  'tuned xgb': [188, 127.9953043460846],
  'tuned knn': [191, 0.7552001476287842]},
 {'nona euclidean': [187, 0.00083160400390625],
  'nona euclidean mlp': [191, 0.5037195682525635],
  'tuned xgb': [189, 126.79080271720886],
  'tuned knn': [194, 0.746436357498169]},
 {'nona euclidean': [189, 0.000911712646484375],
  'nona euclidean mlp': [188, 0.484421968460083],
  'tuned xgb': [182, 120.47955417633057],
  'tuned knn': [190, 0.7459149360656738]},
 {'nona euclidean': [189, 0.0008335113525390625],
  'nona euclidean mlp': [197,

In [42]:
import torch
y_hat = torch.tensor([2.2384e-04, 3.1275e-15, 9.9974e-01, 9.9999e-01, 1.0000e+00, 4.4973e-26,
        4.7756e-02, 4.9139e-19, 1.0000e+00, 1.5738e-07, 9.9145e-01, 3.9661e-01,
        9.9993e-01, 6.0036e-12, 1.0000e+00, 1.0000e+00, 1.0000e+00, 9.9993e-01,
        1.0000e+00, 9.7243e-01, 7.7070e-18, 9.8907e-01, 1.0000e+00, 1.0000e+00,
        9.9921e-01, 4.2330e-05, 1.0000e+00, 3.2234e-13, 9.9927e-01, 4.2973e-10,
        1.0000e+00, 1.2621e-06], dtype=torch.float64)
y = torch.tensor([0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0.], dtype=torch.float64)

In [43]:
loss = torch.nn.BCELoss()
loss(y_hat, y)

tensor(0.0189, dtype=torch.float64)